In [2]:
import os
import orjson
import pandas as pd
import torch
from torchvision.ops import box_convert
import cv2

In [3]:
json_path = 'dataset/test/result/bbox.json'
conf_thresh = 0.8
images = os.listdir('dataset/test/images')
h, w  = 720, 1280
with open(json_path, 'rb') as f:
    json_data = orjson.loads(f.read())
results = []
same_image_results = []
last_seen_img_id = None
for result in json_data:
    if not last_seen_img_id:
        last_seen_img_id = result['image_id']
    img_id = result['image_id']

    if last_seen_img_id and img_id != last_seen_img_id:  # if not None (first run) and a different image
        same_image_results.sort(key=lambda x: x['confidence'], reverse=True)
        results.extend(same_image_results[:4])  # 1 image at most 4 bbox
        same_image_results = []
        last_seen_img_id = result['image_id']
    if result['score'] > conf_thresh:
        result['bbox'] = box_convert(torch.tensor([result['bbox']]), 'xywh', 'xyxy')[0].tolist()  # raw RT-DETR checkpoint uses COCO format which is xywh, convert to xyxy. Note exported RT-DETR uses xyxy
        same_image_results.append({
            'Image_ID': images[int(img_id)].split('.')[0],
            'class': 0,  # TODO: replace with actual ReID model results, but for now can just submit bbox and see map, 1: suspect, 0: not suspect
            'confidence': result['score'],
            'ymin': result['bbox'][1] / h,
            'xmin': result['bbox'][0] / w,
            'ymax': result['bbox'][3] / h,
            'xmax': result['bbox'][2] / w
        })
else:  # when for loop exhausts, do 1 last merge
    same_image_results.sort(key=lambda x: x['confidence'], reverse=True)
    results.extend(same_image_results[:4])  # 1 image at most 4 bbox
    same_image_results = []
assert len(set(r['Image_ID'] for r in results)) == len(images)  # check all image have at least 1 bbox

In [4]:
df = pd.DataFrame(results)
df.to_csv('submission.csv', index=False)